In [15]:
import pandas as pd
articleDetails = pd.read_pickle('article_details.pkl')

In [16]:
articleDetails.head()

,Date,Link,Title,Article Details,Post Time,List Links
0,2022-06-04,https://order-order.com/2022/06/04/saturday-7-...,Saturday 7-Up,"This week 158,742 visitors visited 452,385 tim...",07:35,[EXCLUSIVE: Yvette Cooper’s Son Led Trans Righ...
1,2022-05-28,https://order-order.com/2022/05/28/saturday-7-...,Saturday 7-Up,"This week 192,138 visitors visited 604,322 tim...",07:30,[Jess Phillips Investigation Concludes Multipl...
2,2022-05-21,https://order-order.com/2022/05/21/saturday-7-...,Saturday 7-Up,"This week 201,390 visitors visited 582,067 tim...",07:30,[Andrew Rosindell’s Office Not Answering Queri...
3,2022-05-07,https://order-order.com/2022/05/07/saturday-7-...,Saturday 7-Up,,07:30,[LibDem Candidate’s Pornstar Sideline Revealed...
4,2022-04-30,https://order-order.com/2022/04/30/saturday-7-...,Saturday 7-Up,,07:30,"[Flashback: What They Said At the Time, READ I..."


In [18]:
article_index = 3
print(articleDetails.iloc[[article_index]])
articleDetails.iloc[[article_index]].Link[article_index]

        Date                                               Link  \
3 2022-05-07  https://order-order.com/2022/05/07/saturday-7-...   

           Title Article Details Post Time  \
3  Saturday 7-Up                     07:30   

                                          List Links  
3  [LibDem Candidate’s Pornstar Sideline Revealed...  


'https://order-order.com/2022/05/07/saturday-7-up-179/'